In [16]:
%load_ext autoreload
%autoreload 2

import pickle

from Preprocessing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Cleaning data

Apply the functions in Preprocessing.py to clean the midi dataset. There are multiple files that are currupted. 
For this analysis we are also going to use only Midi file with a timestamp of 4/4, like in the reference paper. This filtering is done in CleaningData()

In [12]:
CleaningData()

Cleaning Data : 100%|██████████| 2200/2200 [10:32<00:00,  3.48it/s]


# Preprocessing data:

Firstly we reconstruct the database, transforming all the polyphonic audios into monophonic, keeping the information about the tracks in the midi file. It is done by keeping only the highest pitch from each polyphonic note.

In [13]:
RecreateDatabase()

Recreating Database : 100%|██████████| 2079/2079 [14:22<00:00,  2.41it/s] 


The input of the model has to be a 128x16 matrix as in the paper. The following function clasify the midi tracks into 7 instrumental classes:
- String
- Keyboard
- Aereophone
- Percussion
- Voice
- Synth
- Others

In [14]:
Dataset = PreProcessing()

Preprocessing : 100%|██████████| 2077/2077 [03:47<00:00,  9.12it/s]


In [ ]:
# with open('Dataset.pkl', 'wb') as f:
#    pickle.dump(Dataset, f)

with open('Dataset.pkl', 'rb') as f:
   Dataset = pickle.load(f)